In [1]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
import weather_forecast_utils
reload(weather_forecast_utils)
from weather_forecast_utils import *

---

In [2]:
def reverse_day(year, day):
    feb = 28
    if year in list(range(2000, 2025, 4)):
        feb += 1
            
    days_per_month = [31, feb, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    for index, month in enumerate(days_per_month):
        if (day - month) == 0:
            return (month, index+1)
        elif (day - month) > 0:
            day -= month
        else:
            return (day, index+1)

In [ ]:
target = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_negative_weather_grid.geojson')
target.isna().sum()

In [ ]:
target['DD'] = target.apply(lambda x: reverse_day(x['YYYY'], x['day'])[0], axis=1)
target['MM'] = target.apply(lambda x: reverse_day(x['YYYY'], x['day'])[1], axis=1)
target.to_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_negative_weather_grid.geojson', index=False)

---

In [ ]:
positives = dict()
negatives = dict()

for y in range(2016, 2025):
    positives[y] = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/output_weather_kriging/{y}.geojson')
    negatives[y] = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/negative_output_weather_kriging/{y}.geojson')

In [ ]:
for y in range(2016, 2025):
    save_clean_data(pd.DataFrame(positives[y]), "positives_"+str(y), '/nfs/home/genovese/thesis-wildfire-genovese/database/cache', force=True)
    save_clean_data(pd.DataFrame(negatives[y]), "negatives_"+str(y), '/nfs/home/genovese/thesis-wildfire-genovese/database/cache', force=True)

In [ ]:
merged = dict()

positives = dict()
negatives = dict()

for y in range(2016, 2025):
    positives[y] = pd.read_csv(f'/nfs/home/genovese/thesis-wildfire-genovese/database/cache/positives_{y}.csv')
    negatives[y] = pd.read_csv(f'/nfs/home/genovese/thesis-wildfire-genovese/database/cache/negatives_{y}.csv')

for y in positives.keys():
    merged[y] = pd.concat([positives[y], negatives[y]], ignore_index=True)

In [ ]:
from shapely import wkt

for y, gdf in merged.items():
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
    save_clean_data(gpd.GeoDataFrame(gdf, geometry='geometry', crs="EPSG:3857"), f'meteo_{y}', '/nfs/home/genovese/thesis-wildfire-genovese/database',
                    force=True)

---